In [ ]:
# IDEAS:
# preguntas sugeridas para candidato

In [1]:
import os
import utils.cv_reviewer_utils as cv_utils
from IPython.display import Markdown, display
import json

In [2]:
#if its published somewhere
#job_description = get_job_description("url")

job_description = """

Job Title: Junior Data Engineer
Location: Remote
Job Type: Full-time
Team: Data Engineering
Reports to: Senior Data Engineer

About the Role:

We are looking for a Junior Data Engineer to join our growing data team. You will support the design, development, and maintenance of data pipelines and infrastructure that power data analytics, reporting, and data-driven decision-making across the company.
This is a great opportunity for someone early in their career to learn and grow while working on real-world data challenges in a collaborative and agile environment.

Key Responsibilities
- Assist in building, maintaining, and optimizing ETL/ELT pipelines.
- Work closely with data analysts, scientists, and engineers to support data ingestion and transformation workflows.
- Help ensure the reliability and quality of data across systems.
- Monitor and troubleshoot data pipelines and processes.
- Contribute to documentation of data flows, models, and architecture.
- Learn and apply best practices in data engineering, including security and scalability.

Requirements:

- Basic knowledge of SQL and at least one programming language (Python preferred).
- Familiarity with data storage systems (e.g., relational databases, cloud storage).
- Understanding of data processing concepts and tools.
- Willingness to learn and grow in a fast-paced environment.
- Good communication and problem-solving skills.
- Bachelor’s degree in Computer Science, Engineering, Mathematics, or related field (or equivalent practical experience).

Nice to Have:

- Exposure to cloud platforms like AWS, GCP, or Azure.
- Experience with version control (e.g., Git).
- Basic knowledge of data modeling concepts.
- Familiarity with tools like Airflow, dbt, or Spark.

"""

# in lower case
mandatory_keywrods = ["spark","python"]



In [5]:
landing_path = './landing'
        
# "llama3.2" or chatgpt model
model = "llama3.2"

descriptions_dict = {}
evaluation_dict = {}
matches = []

for filename in os.listdir(landing_path):
    path = os.path.join(landing_path, filename)
    
    if os.path.isfile(path):
        cv_text = cv_utils.extract_text_from_cv(path)
        if cv_text is not None:
            
            words = cv_text.split()    
            num_of_words = len(words)

            if num_of_words > 5:    
                anonymized_desc = cv_utils.anonymize_resume(cv_text)

                # keywords string is not mandatory
                keywords_string = "Additional note: " + cv_utils.evaluate_mandatory_keywords(anonymized_desc,mandatory_keywrods)

                llm_answer = cv_utils.evaluate_candidate(model, anonymized_desc, job_description, keywords_string)

                print(llm_answer)
                llm_answer['name'] = filename

                updated_json_str = json.dumps(llm_answer, indent=2)

                matches.append(updated_json_str)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using llama3.2 via Ollama...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


{'name': 'Data Engineer B.A.', 'match_percentage': 60, 'summary': 'The candidate has a strong background in data engineering with experience in Spark, Python, and AWS. However, the seniority does not match the job description as they are currently working as Data Engineers and do not mention junior or any lower position.'}
▶️ Using llama3.2 via Ollama...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


{'name': 'Anonymized CV Holder', 'match_percentage': 50, 'summary': 'Lacks seniority and cloud experience, has some relevant skills like Spark, Python, and database systems. Gaps in security and scalability knowledge.'}
▶️ Using llama3.2 via Ollama...


CropBox missing from /Page, defaulting to MediaBox


{'name': 'Anonymous Data Engineer', 'match_percentage': 32, 'summary': 'Although the candidate has experience with large datasets and some programming languages (Java, Python), their seniority and match to the Junior Data Engineer role are limited. The candidate lacks direct experience with ETL/ELT pipelines, data storage systems, and cloud platforms, which are essential for the job. They also lack knowledge of tools like Airflow, dbt, or Spark, which is a nice-to-have requirement.'}


CropBox missing from /Page, defaulting to MediaBox


▶️ Using llama3.2 via Ollama...
{'name': 'Thaddeus', 'match_percentage': 70, 'summary': 'Strong knowledge in AWS, Python and MySQL. Experience with data pipelines on Airflow and ETL workflows using Apache NiFi. However, lacks experience with Spark and cloud platforms like GCP or Azure.'}


In [8]:
# Convierte elementos si son strings
parsed_matches = [json.loads(m) if isinstance(m, str) else m for m in matches]

# Ahora puedes ordenarlos sin error
sorted_matches = sorted(parsed_matches, key=lambda x: x['match_percentage'], reverse=True)

markdown_text = """
### 📊 Candidate Evaluations

"""

n = 1
for match in sorted_matches:
    markdown_text += f"""**🧑‍💼 Candidate nº{n}: {match['name']} – {match['match_percentage']}%**

    {match['summary']}
    
"""
    n+=1

display(Markdown(markdown_text))


### 📊 Candidate Evaluations

**🧑‍💼 Candidate nº1: Thaddeus Drake.pdf – 70%**

    Strong knowledge in AWS, Python and MySQL. Experience with data pipelines on Airflow and ETL workflows using Apache NiFi. However, lacks experience with Spark and cloud platforms like GCP or Azure.
    
**🧑‍💼 Candidate nº2: alan susa.pdf – 60%**

    The candidate has a strong background in data engineering with experience in Spark, Python, and AWS. However, the seniority does not match the job description as they are currently working as Data Engineers and do not mention junior or any lower position.
    
**🧑‍💼 Candidate nº3: Elara Quinn.pdf – 50%**

    Lacks seniority and cloud experience, has some relevant skills like Spark, Python, and database systems. Gaps in security and scalability knowledge.
    
**🧑‍💼 Candidate nº4: john smith.pdf – 32%**

    Although the candidate has experience with large datasets and some programming languages (Java, Python), their seniority and match to the Junior Data Engineer role are limited. The candidate lacks direct experience with ETL/ELT pipelines, data storage systems, and cloud platforms, which are essential for the job. They also lack knowledge of tools like Airflow, dbt, or Spark, which is a nice-to-have requirement.
    
